# Data Setting & Save

In [1]:
import sys, os
import pandas as pd
import pathSetting as setting
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

from Clust.app import data_manipulation, data_preprocessing
from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
from Clust.clust.ingestion.mongo.mongo_client import MongoClient

db_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)
mongo_client = MongoClient(ins.CLUSTMetaInfo2)


## Setting

In [2]:
bucket_name = 'integration'

data_mode_list =['trainX', 'trainy', 'testX', 'testy']
clean_level_list =[0, 4]
clean_level = clean_level_list[0] # 현재 classification은 clean level 0만 사용

trainStartTime = "1992-1-11"
trainEndTime = "2012-02-27"
testStartTime = "2013-1-1"
testEndTim = "2016-10-16"

process_param = data_manipulation.get_process_param_by_level(clean_level)

for data_mode in data_mode_list:
    ingestion_param ={}
    integration_param = {
        #"integration_frequency":integration_freq_sec,
        "param":{},
        "method":"meta",
        "integration_duration":"common"
    }
    if data_mode == 'trainX':
        ingestion_param['start_time'] = trainStartTime
        ingestion_param['end_time'] = trainEndTime
        ingestion_param['ms_list_info'] = [['bio_action_sensors', 'accelerationTrain']]
        integration_param["integration_frequency"] = 60 * 11 + 15 # 11분 15초
    elif data_mode =='testX':
        ingestion_param['start_time'] = testStartTime
        ingestion_param['end_time'] = testEndTim
        ingestion_param['ms_list_info'] = [['bio_action_sensors', 'accelerationTest']]
        integration_param["integration_frequency"] = 60 * 11 + 15 # 11분 15초
    elif data_mode == 'trainy':
        ingestion_param['start_time'] = trainStartTime
        ingestion_param['end_time'] = trainEndTime
        ingestion_param['ms_list_info'] = [['life_action_pattern', 'patternNameTrain']]
        integration_param["integration_frequency"] = 60 * 60 * 24 # 1일
    elif data_mode =='testy':
        ingestion_param['start_time'] = testStartTime
        ingestion_param['end_time'] = testEndTim
        ingestion_param['ms_list_info'] = [['life_action_pattern', 'patternNameTest']]
        integration_param["integration_frequency"] = 60 * 60 * 24 # 1일


    data = data_manipulation.ingestion_processing_integration(db_client, ingestion_param, process_param, integration_param)
    ms_name = "classification_actionPattern_"+ data_mode + '_cleanLevel' + str(clean_level)
    db_client.write_db(bucket_name, ms_name, data)

    # save to mongodb
    meta_info={"data_name": ms_name,"ingestion_param": ingestion_param, 
               "integration_param":integration_param, "clean_level":clean_level, "process_param":process_param}
    collection_name = "classification_actionPattern"
    mongo_client.insert_document(bucket_name, collection_name, meta_info)
    print(meta_info)

===integrationStart===
===integrationEnd===
========== write success ==========
========== Data Save Success ==========
{'data_name': 'classification_actionPattern_trainX_cleanLevel0', 'ingestion_param': {'start_time': '1992-1-11', 'end_time': '2012-02-27', 'ms_list_info': [['bio_action_sensors', 'accelerationTrain']]}, 'integration_param': {'param': {}, 'method': 'meta', 'integration_duration': 'common', 'integration_frequency': 675}, 'clean_level': 0, 'process_param': {'refine_param': {'removeDuplication': {'flag': False}, 'staticFrequency': {'flag': False, 'frequency': None}}, 'outlier_param': {'certainErrorToNaN': {'flag': False}, 'unCertainErrorToNaN': {'flag': False}}, 'imputation_param': {'flag': False}}, '_id': ObjectId('643f5f032a442133e8cca965')}
===integrationStart===
===integrationEnd===
========== write success ==========
========== Data Save Success ==========
{'data_name': 'classification_actionPattern_trainy_cleanLevel0', 'ingestion_param': {'start_time': '1992-1-11', '